In [ ]:
import time
import csv
import re
import requests
import json
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import *

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument("disable-gpu")

driver_path = 'D:/JISUNG/jupyter/chromedriver/chromedriver.exe'
browser = webdriver.Chrome(driver_path, chrome_options=options)
browser2 = webdriver.Chrome(driver_path, chrome_options=options)
browser.get('http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do')
count = int(browser.find_element_by_xpath('//*[@id="content"]/div[4]/div[1]/span/em').text.replace(',', ''))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/a[1]'))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="sPrdtStatStr"]'))
while 1:
    try:
        browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="mul_chk_det0"]'))
        break
    except:
        pass
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="layerConfirmChk"]'))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="searchForm"]/div[1]/div[5]/button[1]'))
while 1:
    try:
        browser.find_element_by_xpath('//*[@class="pageLoading"]')
    except:
        break
while 1:
    max_count = int(browser.find_element_by_xpath('//*[@id="content"]/div[4]/div[1]/span/em').text.replace(',', ''))
    if count != max_count:
        break
file = open('movie.csv', 'wt', encoding='utf-8', newline='')
writer = csv.writer(file)
writer.writerow(
    [
        'movie_id', # 영화 코드
        'title', # 제목
        'audience', # 관객수(종속변수)
        'release_date', # 개봉일
        'country', # 제작 국가
        'director', # 감독
        'producer', # 제작사
        'actor1', # 주연1
        'actor2', # 주연2
        'actor3', # 주연3
        'genre', # 장르
        'screen', # 스크린 수
        'running_time', # 상영시간
        'rating', # 등급
        'series', # 시리즈
        'holiday', # 개봉일부터 한 달 동안의 휴일 수
    ]
)

genres = ['드라마', '코미디', '액션', '멜로/로맨스', '스릴러', '미스터리',
          '공포(호러)', '어드벤처', '범죄', '가족', '판타지', 'SF',
          '서부극(웨스턴)', '사극', '애니메이션', '다큐멘터리', '전쟁',
          '뮤지컬', '성인물(에로)', '공연', '기타']

page = 1
for i in range(1, max_count + 1):
    try: 
        page_check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
        movie_id = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[3]/span').text.strip()
        title = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[1]/span/a').text.strip()
        temp = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[5]/span').text.strip()
        country = re.compile(r'[가-힣]+').search(temp).group()
        director = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[9]/span/a').text.strip()
        producer = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[10]/span/a').text.strip()
        temp = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[7]/span').text.strip()
        
        browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[1]/span/a'))
        while 1:                                  
            try:
                browser.find_element_by_xpath('/html/body/div[4]')
                break
            except:
                pass
        
        browser2.get('https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=')
        element = browser2.find_element_by_xpath('//*[@id="nx_query"]')
        element.send_keys(title)
        element.submit()
        try:
            series = ''
            browser2.find_element_by_xpath('//*[@id="_movie_tab"]/div/ul/li[4]/a')
            parent = browser2.find_element_by_xpath('//*[@id="_movie_tab_content_series"]/div[1]')
            child = parent.find_elements_by_tag_name('ul')
            for ul in child:
                for li in ul.find_elements_by_tag_name('li'):
                    for dl in li.find_elements_by_tag_name('dl'):
                        series_title = dl.find_element_by_xpath('./dt[1]/a').get_attribute('textContent')
                        if title != series_title:
                            series += series_title + ','
        except NoSuchElementException:
            print(f'- {title}|시리즈 없음')
        except Exception as e:
            print(f'[ERROR] {i}번|{movie_id}|{title}\n{type(e)}\n{e}')
        
        if browser.find_element_by_xpath('//*[@class="ovf cont"]/dd[4]').text == '':
            temp = browser.find_element_by_xpath('//*[@class="ovf cont"]').get_attribute('textContent').replace(' ', '').replace('\n', '').replace('\t', '').split('|')
            for j in temp:
                if '분' in j:
                    running_time = j.split('분')[0]
                elif '관람' in j:
                    rating = j
                else:
                    for k in genres:
                        if k in j:
                            genre = j.replace(' ', '')
                            break
            
            release_date = re.compile(r'[0-9]{4}\-[0-9]{2}\-[0-9]{2}').search(temp[5]).group()
            
            try:
                temp = browser.find_element_by_xpath(f'//*[@class="staffMore"]/dl/div[2]').get_attribute('textContent').replace('배우[주연]', '').split('|')

                if len(temp) > 3:
                    for j in range(1, 4):
                        exec(f'actor{j} = temp[{j}].split()')
                else:
                    for j in range(1, len(temp) + 1):
                        exec(f'actor{j} = temp[{j}].split()')
                    for j in range(len(temp) + 1, 4):
                        exec(f'actor{j} = ""')
            except NoSuchElementException:
                print(f'- {title}|주연 정보 누락')
                actor1 = actor2 = actor3 = ''
            except Exception as e:
                print(f'[ERROR] {i}번|주연 정보 오류|{movie_id}|{title}\n{type(e)}\n{e}')

            browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@class="wrap_tab tab2"]/ul/li[2]'))
            while 1:
                try:
                    browser.find_element_by_xpath(f'//*[@class="item_tab statistics"]/p')
                except:
                    break

            try:
                temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[2]').get_attribute('textContent')
                screen = re.compile(r'\d+\n').search(temp).group().strip()
            except NoSuchElementException:
                print(f'- {title}|스크린수 정보 누락')
                screen = ''
            try:
                temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[4]').get_attribute('textContent')
                audience = re.compile(r'[\d]+\s').search(temp.replace(',', '')).group().strip()
            except NoSuchElementException:
                print(f'- {title}|관객수 정보 누락')
                audience = ''

        else:
            temp = browser.find_element_by_xpath('//*[@class="ovf cont"]/dd[4]').text.split('|')
            for j in temp:
                if '분' in j:
                    running_time = j.strip().split('분')[0]
                elif '관람' in j:
                    rating = j.strip()
                else:
                    for k in genres:
                        if k in j:
                            genre = j.replace(' ', '').replace(',', '|')
                            break
            
            if browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dt[5]').text.strip() == '개봉일':
                release_date = browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dd[5]').text.strip()
            else:
                release_date = browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dd[6]').text.strip()
            for j in range(1, 4):
                try:
                    if browser.find_element_by_xpath(f'//*[@class="staffMore"]/dl/div[2]/dt').text.strip() == '배우':
                        try:
                            temp = f'browser.find_element_by_xpath(\'//*[@class="staffMore"]/dl/div[2]/dd/table[1]/tbody/tr/td/a[{j}]\').text.strip()'
                            t = browser.find_element_by_xpath(f'//*[@class="staffMore"]/dl/div[2]/dd/table[1]/tbody/tr/td/a[{j}]').text.strip()
                            if re.compile(r"\(").search(t) == None:
                                exec(f'actor{j} = {temp}')
                            else:
                                exec(f'actor{j} = re.compile(r"[가-힣]+").search({temp}).group().strip()')
                        except NoSuchElementException:
                            exec(f'actor{j} = actor{j+1} = actor{j+2} = ""')
                            break
                        except Exception as e:
                            print(f'[ERROR] {i}번|주연 정보 오류|{movie_id}|{title}\n{type(e)}\n{e}')
                    else:
                        print(f'- {title}|주연 정보 누락')
                        actor1 = actor2 = actor3 = ''
                        break
                except NoSuchElementException:
                    print(f'- {title}|주연 정보 누락')
                    actor1 = actor2 = actor3 = ''
                    break
                except Exception as e:
                    print(f'[ERROR] {i}번|주연 정보 오류|{movie_id}|{title}\n{type(e)}\n{e}')

            browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@class="wrap_tab tab2"]/ul/li[2]'))
            while 1:
                try:
                    browser.find_element_by_xpath(f'//*[@class="item_tab statistics"]/p')
                except:
                    break

            try:
                temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[2]').text
                screen = re.compile(r'\d+\n').search(temp).group().strip()
            except NoSuchElementException:
                print(f'- {title}|스크린수 정보 누락')
                screen = ''
            try:
                temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[4]').text
                audience = re.compile(r'[\d]+\s').search(temp.replace(',', '')).group().strip()
            except NoSuchElementException:
                print(f'- {title}|관객수 정보 누락')
                audience = ''

        browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@class="hd_layer"]/a[2]'))

        holiday = 0
        if '없음' not in release_date:
            temp = release_date.split('-')
            year = temp[0]
            month = temp[1]
            day = temp[2]
            date = datetime.date(int(year), int(month), int(day))


            for _ in range(31):
                d = (date).weekday()

                if date.year == int(year) and date.month == int(month):
                    url = f'https://m.search.naver.com/p/csearch/content/qapirender.nhn?where=nexearch&key=CalendarAnniversary&pkid=134&q={year}{month}%EC%9B%94&_callback=window.__jindo2_callback._directsearch_calendar_famous_month_0'
                else:
                    url = f'https://m.search.naver.com/p/csearch/content/qapirender.nhn?where=nexearch&key=CalendarAnniversary&pkid=134&q={date.year}{"0" + str(date.month) if len(str(date.month)) == 1 else str(date.month)}%EC%9B%94&_callback=window.__jindo2_callback._directsearch_calendar_famous_month_0'

                response = requests.get(url)
                soup = BeautifulSoup(response.text, 'html.parser')
                data = re.compile(r'[(](.*?)[)]').search(str(soup)).group().replace('(', '').replace(')', '')
                parse = json.loads(data)

                day_off = 0
                for k in parse['openCalendar']['daysList']:
                    if k['solarDate'] == str(date).replace('-', ''):
                        if k['dayOff'] == 'true':
                            day_off = 1
                            break

                if d == 5 or d == 6 or day_off:
                    holiday += 1
                date += datetime.timedelta(days=1)
        
        print(f'{i}번 {movie_id}, {title}, {audience}, {release_date}, {country}, {director}, {producer}, {actor1}, {actor2}, {actor3}, {genre}, {screen}, {running_time}, {series}, {rating}, {holiday}')
        
        writer.writerow(
            [
                movie_id, 
                title,
                audience,
                release_date,
                country,
                director,
                producer,
                actor1,
                actor2,
                actor3,
                genre,
                screen,
                running_time,
                rating,
                series[:-1],
                holiday
            ]
        )
    except Exception as e:
        print(f'[ERROR!] {i}번|{movie_id}|{title}\n{type(e)}\n{e}')
    finally:
        print('')
        try:
            if not i % 10:
                if page % 10:
                    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@id="pagingForm"]/div/ul/li[{10 if page % 10 == 9 else (page + 1) % 10}]/a'))
                else:
                    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="pagingForm"]/div/a[3]'))
                page += 1
                while 1:
                    try:
                        check = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[1]/td[1]/span/a').text.strip()
                        if page_check != check:
                            break
                    except:
                        pass
        except Exception as e:
            print(f'[ERROR!!] {i}번|{movie_id}|{title}\n{type(e)}\n{e}')
    
file.close()
browser.quit()
browser2.quit()

C:\Users\jejsm\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()
C:\Users\jejsm\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


- 딥워터|시리즈 없음
1번 20209685, 딥워터, 7331, 2020-07-09, 기타, 요아힘 헤덴, , 모아 감멜, 매들린 마틴, , 기타, 264, 81, , 12세이상관람가, 9

- 와일드 시티|시리즈 없음
2번 20209408, 와일드 시티, 4163, 2020-06-17, 영국, 론 스캘펠로, , 샘 클라플린, 티모시 스폴, 노엘 클락, 액션, 111, 102, , 청소년관람불가, 8

- 지옥의 묵시록|시리즈 없음
3번 19790164, 지옥의 묵시록, 550, 1988-06-04, 미국, 프란시스 포드 코폴라, , 말론 브란도, 마틴 쉰, 로버트 듀발, 전쟁|드라마, 1, 147, , 고등학생이상관람가, 11

- 지옥의 묵시록 리덕스|시리즈 없음
4번 20010061, 지옥의 묵시록 리덕스, 2808, 2001-08-30, 미국, 프란시스 포드 코폴라, , 로버트 듀발, 말론 브란도, 마틴 쉰, 전쟁|드라마, 7, 199, , 18세관람가, 9

- 16세의 사운드트랙|시리즈 없음
5번 20208887, 16세의 사운드트랙, 388, 2020-07-02, 영국, 힐러리 셰익스피어, , 제임스 캘로웨이, 스칼렛 마셜, , 코미디|멜로/로맨스, 19, 84, , 12세이상관람가, 9

- 트랜짓|시리즈 없음
6번 20196938, 트랜짓, 3558, 2020-07-02, 독일, 크리스티안 펫졸드, , 프란츠 로고스키, 폴라 비어, , 드라마|SF, 42, 101, , 12세이상관람가, 9

- 프랑스여자|시리즈 없음
7번 20190461, 프랑스여자, 18352, 2020-06-04, 한국, 김희정, (주)인벤트스톤, 김호정, 김지영, 김영민, 드라마, 244, 88, , 15세이상관람가, 9

- 도미노|시리즈 없음
8번 20209362, 도미노, 2730, 2020-06-11, 덴마크, 브라이언 드 팔마, , 니콜라이 코스터 왈도, 캐리스 밴 허슨, 가이 피어스, 범죄|스릴러, 113, 89, , 15세이상관람가, 9

- 모탈: 레전드